In [1]:
import pandas as pd 

ukrainian_tweets = pd.read_csv('data/ukrainian_only/ukrainian_tweets.csv')
ukrainian_tweets.shape

/tmp/ipykernel_31961/1232358823.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/tmp/ipykernel_31961/1232358823.py:3: DtypeWarning: Columns (0,1,6,7,8,10,16,17,18,20,21,25,28) have mixed types. Specify dtype option on import or set low_memory=False.
  ukrainian_tweets = pd.read_csv('data/ukrainian_only/ukrainian_tweets.csv')


(1490795, 30)

### drop rows with empty text and duplicates

In [2]:
ukrainian_tweets = ukrainian_tweets.dropna(subset=['text'])
ukrainian_tweets = ukrainian_tweets.drop_duplicates(subset=['text'])
ukrainian_tweets['text'] = ukrainian_tweets.text.apply(lambda x : x.lower())
ukrainian_tweets.shape

(779836, 30)

### Remove stopwords and punctuation

In [3]:
with open('stopwords_ua.txt', 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

stopwords[:5]

['а', 'аби', 'абиде', 'абиким', 'абикого']

In [4]:
"це" in stopwords

True

In [5]:
import re

def remove_stopwords_and_punctuation(text : str):
    """Split text by space char and remove stopwords"""

    text_no_punctuation = re.sub(r'[^\w\s]', '', text)
    words = text_no_punctuation.split()  # Split text into words
    filtered_words = [word for word in words if word not in stopwords]  # Keep words that are not in stopwords
    return ' '.join(filtered_words)  # Join words back into a string

In [6]:
ukrainian_tweets['text_processed'] =  ukrainian_tweets['text'].apply(remove_stopwords_and_punctuation)

In [7]:
ukrainian_tweets['text_processed']

0          urijah_ghor xaviercrab шо федора комікса 7 кар...
1          польща заборонила ввозити україни десятки найм...
2          вкажіть вік опублікувавши гру вийшла 18 httpst...
3          україна закупила обладнання відбору зберігання...
4          baldreview оффтоп взагалі обурює ліниві вибляд...
                                 ...                        
1490790                                         coricciv бля
1490791                     емоційно узагальнила здвиги фазі
1490792               charlotta_cos швидкого мовити одужання
1490793    ідіть ебать путін ukrainian putinswar putinwar...
1490794    запаси 1 2 3 4 груп крові закінчились повідомл...
Name: text_processed, Length: 779836, dtype: object

In [14]:
ukrainian_tweets = ukrainian_tweets[~ukrainian_tweets['text_processed'].isna()]

### remove emoji

In [15]:
try:
    import emoji
except ModuleNotFoundError:
    !python -m pip install emoji --upgrade
    import emoji
    
def give_emoji_free_text(text):
    return emoji.replace_emoji(text,replace='')

In [18]:
ukrainian_tweets['text_processed'] =  ukrainian_tweets['text_processed'].apply(lambda x : give_emoji_free_text(x))

### remove english characters ?

In [19]:
def remove_english_characters(text):
    return re.sub(r'[a-zA-Z]', '', text)

In [20]:
ukrainian_tweets['text_processed'] = ukrainian_tweets['text_processed'].apply(remove_english_characters)

In [21]:
ukrainian_tweets['text_processed']

0          _  шо федора комікса 7 карт знаю чесно шось за...
1          польща заборонила ввозити україни десятки найм...
2              вкажіть вік опублікувавши гру вийшла 18 39 04
3          україна закупила обладнання відбору зберігання...
4           оффтоп взагалі обурює ліниві виблядки кріслах...
                                 ...                        
1490790                                                  бля
1490791                     емоційно узагальнила здвиги фазі
1490792                           _ швидкого мовити одужання
1490793                               ідіть ебать путін     
1490794    запаси 1 2 3 4 груп крові закінчились повідомл...
Name: text_processed, Length: 779836, dtype: object

### remove numbers and remove extra spaces

In [22]:
def remove_numbers(text):
    return re.sub(r'\d+', '', text)

In [23]:
def remove_extra_spaces(text):
    text = re.sub(r'\s+', ' ', text).strip()
    return text.replace('_','') 

In [24]:
ukrainian_tweets['text_processed'] = ukrainian_tweets['text_processed'].apply(remove_numbers)

In [25]:
ukrainian_tweets['text_processed'] 

0          _  шо федора комікса  карт знаю чесно шось зай...
1          польща заборонила ввозити україни десятки найм...
2                    вкажіть вік опублікувавши гру вийшла   
3          україна закупила обладнання відбору зберігання...
4           оффтоп взагалі обурює ліниві виблядки кріслах...
                                 ...                        
1490790                                                  бля
1490791                     емоційно узагальнила здвиги фазі
1490792                           _ швидкого мовити одужання
1490793                               ідіть ебать путін     
1490794    запаси     груп крові закінчились повідомляє х...
Name: text_processed, Length: 779836, dtype: object

In [26]:
ukrainian_tweets['text_processed'] = ukrainian_tweets['text_processed'].apply(remove_extra_spaces)

In [27]:
ukrainian_tweets['text_processed']

0           шо федора комікса карт знаю чесно шось зайшло...
1          польща заборонила ввозити україни десятки найм...
2                       вкажіть вік опублікувавши гру вийшла
3          україна закупила обладнання відбору зберігання...
4          оффтоп взагалі обурює ліниві виблядки кріслах ...
                                 ...                        
1490790                                                  бля
1490791                     емоційно узагальнила здвиги фазі
1490792                             швидкого мовити одужання
1490793                                    ідіть ебать путін
1490794    запаси груп крові закінчились повідомляє харкі...
Name: text_processed, Length: 779836, dtype: object

### lemmatization

In [24]:
try :
    import pymorphy3
except ModuleNotFoundError:
    ! pip install pymorphy3
    ! pip install pymorphy3-dicts-uk
    import pymorphy3
except Exception as e:
    print(f"Error occured during importing pymorphy {e}")
    

  Obtaining dependency information for pymorphy3 from https://files.pythonhosted.org/packages/ee/53/862f7b7f3e488e5420bebd5cf59362cb175463ad3cfddd61ade15a738dc7/pymorphy3-2.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for pymorphy3-dicts-ru from https://files.pythonhosted.org/packages/b0/67/469e9e52d046863f5959928794d3067d455a77f580bf4a662630a43eb426/pymorphy3_dicts_ru-2.4.417150.4580142-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 965.5 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 6.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [30]:
# Create a morphological analyzer for Ukrainian
morph = pymorphy3.MorphAnalyzer(lang='uk')


def lemmatize_text_uk(text):
    words = text.split()  
    lemmatized_words = [morph.parse(word)[0].normal_form for word in words]
    return ' '.join(lemmatized_words)

In [31]:
ukrainian_tweets['text_processed'] = ukrainian_tweets['text_processed'].apply(lemmatize_text_uk) 

In [32]:
ukrainian_tweets['text_processed'] 

0          _ здаватися шо федора комікс карта знати чесно...
1          польща заборонити ввозити україна десятка найм...
2                       вказати віка опублікувавши гру вийти
3          україна закупити обладнання відбір зберігання ...
4          та оффтоп взагалі мен обурювати лінивий вибляд...
                                 ...                        
1490790                                               та бля
1490791                      емоційно узагальнити здвиг фаза
1490792                         _ це швидкий мовити одужання
1490793                                      іти ебати путін
1490794    запас група кров закінчитися повідомляти харкі...
Name: text_processed, Length: 779836, dtype: object

In [ ]:
ukrainian_tweets.to_csv('./data/processed/ukrainian_tweets_processed.csv',index=False)